In [5]:
import subprocess

def get_ffmpeg_version():
    try:
        # 使用subprocess模块调用ffmpeg -version命令
        result = subprocess.run(['ffmpeg', '-version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        # 解码输出结果
        output = result.stdout.decode('utf-8')
        # 获取版本信息（输出的第一行）
        version_line = output.split('\n')[0]
        # 打印版本信息
        print(f"Current FFmpeg version: {version_line}")
    except Exception as e:
        print(f"Error: {e}")

# 调用函数打印FFmpeg版本
get_ffmpeg_version()


Current FFmpeg version: ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers


In [6]:
import subprocess

def check_iframe_consistency(video_path):
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-show_frames',
        '-show_entries', 'frame=pict_type',
        '-of', 'csv',
        video_path
    ]
    
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    
    # 解析输出，记录I帧的位置
    iframe_positions = []
    frame_count = 0
    
    for line in result.stdout.splitlines():
        if 'I' in line:
            iframe_positions.append(frame_count)
        frame_count += 1
    
    # 检查I帧间隔是否一致
    if len(iframe_positions) > 1:
        intervals = [iframe_positions[i] - iframe_positions[i-1] for i in range(1, len(iframe_positions))]
        average_interval = sum(intervals) / len(intervals)
        consistent = all(interval == intervals[0] for interval in intervals)
        return average_interval, consistent, intervals
    else:
        return None, False, []
#check
video_path = r'D:\小狗视频\2月\20240202kt\020500.mp4'
average_interval, consistent, intervals = check_iframe_consistency(video_path)

if consistent:
    print(f"I帧的出现频率是固定的，平均间隔为 {average_interval} 帧。")
else:
    print(f"I帧的出现频率不是固定的。平均间隔为 {average_interval} 帧，具体间隔为: {intervals}")



I帧的出现频率是固定的，平均间隔为 20.0 帧。


In [7]:
import subprocess

def check_iframe_frequency(video_path, expected_interval):
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-show_frames',
        '-show_entries', 'frame=pict_type',
        '-of', 'csv',
        video_path
    ]
    
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    
    # 解析输出，记录I帧的位置
    iframe_positions = []
    frame_count = 0
    
    for line in result.stdout.splitlines():
        if 'I' in line:
            iframe_positions.append(frame_count)
        frame_count += 1
    
    # 检查I帧间隔是否一致且为预期的间隔
    if len(iframe_positions) > 1:
        intervals = [iframe_positions[i] - iframe_positions[i-1] for i in range(1, len(iframe_positions))]
        consistent = all(interval == expected_interval for interval in intervals)
        return consistent, intervals
    else:
        return False, []

# check 
video_path = r'D:\小狗视频\2月\20240202kt\020500.mp4'
expected_interval = 20  # 预期的I帧间隔为20帧
consistent, intervals = check_iframe_frequency(video_path, expected_interval)

if consistent:
    print(f"I帧的出现频率是固定的，每{expected_interval}帧出现一个I帧。")
else:
    print(f"I帧的出现频率不是固定的。具体间隔为: {intervals}")


I帧的出现频率是固定的，每20帧出现一个I帧。


In [6]:

#有问题
# def extract_frames_by_time(input_path, output_path, start_time, duration, frame_rate):
#     # 计算起始帧和结束帧
#     start_frame = int(start_time * frame_rate)
#     end_frame = int((start_time + duration) * frame_rate) - 1
    
#     # 构建FFmpeg命令
#     command = [
#         'ffmpeg',
#         '-i', input_path,
#         '-vf', f"select='between(n\\,{start_frame}\\,{end_frame})'",
#         '-vsync', 'vfr',
#         output_path
#     ]
    
#     # 执行FFmpeg命令
#     subprocess.run(command, check=True)
    
#     print(f"视频片段已保存为: {output_path}")

# # 示例用法
# input_video_path = r'D:\小狗视频\2月\20240202kt\020500.mp4'
# output_video_path = r'D:\小狗视频\2月\20240202kt_trial\020500_segment.mp4'
# start_time = 9  # 从第9秒开始
# duration = 4    # 持续4秒
# frame_rate = 20 # 视频帧率

# extract_frames_by_time(input_video_path, output_video_path, start_time, duration, frame_rate)


视频片段已保存为: D:\小狗视频\2月\20240202kt_trial\020500_segment.mp4


In [8]:
import subprocess

def get_total_frames(video_path):
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-count_frames',
        '-show_entries', 'stream=nb_read_frames',
        '-print_format', 'default=nokey=1:noprint_wrappers=1',
        video_path
    ]
    
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    total_frames = int(result.stdout.strip())
    return total_frames

# 示例用法
video_path = r'D:\小狗视频\2月\20240202kt\020500.mp4'
total_frames = get_total_frames(video_path)
print(f"视频总帧数: {total_frames}")


视频总帧数: 1200


In [9]:
import subprocess

def get_iframe_positions(video_path):
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-show_frames',
        '-show_entries', 'frame=pict_type,pkt_pts_time',
        '-print_format', 'csv'
    ]
    
    result = subprocess.run(command + [video_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    
    iframe_positions = []
    frame_count = 0
    
    for line in result.stdout.splitlines():
        if 'I' in line:
            iframe_positions.append(frame_count)
        frame_count += 1
    
    return iframe_positions

# 示例用法
video_path = r'D:\小狗视频\2月\20240202kt\020500.mp4'
iframe_positions = get_iframe_positions(video_path)
print(f"I帧位于以下帧位置: {iframe_positions}")


I帧位于以下帧位置: [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500, 520, 540, 560, 580, 600, 620, 640, 660, 680, 700, 720, 740, 760, 780, 800, 820, 840, 860, 880, 900, 920, 940, 960, 980, 1000, 1020, 1040, 1060, 1080, 1100, 1120, 1140, 1160, 1180]


In [ ]:
import subprocess

# 定义输入文件和输出文件路径
input_file = "input.mp4"
output_file = "output.mp4"

# 定义FFmpeg命令
ffmpeg_command = [
    "ffmpeg",
    "-ss", "00:00:09.05",    # 开始时间
    "-i", input_file,        # 输入文件
    "-vf", "fps=20",         # 输出帧率
    "-frames:v", "80",       # 截取80个帧
    "-c:v", "libx264",       # 视频编码器
    "-c:a", "aac",           # 音频编码器
    "-strict", "experimental",  # 允许使用实验性编码器（如果需要）
    output_file              # 输出文件
]

# 运行FFmpeg命令
subprocess.run(ffmpeg_command)


In [ ]:
ffmpeg
-ss 9.05 
-i input.mp4 
-t 4 
-c:v libx264 
-crf 18 -preset fast 
-an 
-force_key_frames "expr:gte(t,n_forced*0.5)" 
final_output.mp4


In [14]:
# Define the input file path
input_file = r'D:\小狗视频\2月\20240202kt\020500.mp4'
# Define the start time, duration, frame rate, and output file path
start_time = 9.05  # start time in seconds
duration = 4  # duration in seconds
frame_rate = 20  # desired frame rate
output_file = r'D:\小狗视频\2月\20240202kt_trial\final_output.mp4'

In [15]:
import subprocess

def precise_clip(input_file, start_time, duration, output_file, frame_rate):
    # Construct the FFmpeg command
    ffmpeg_command = [
        "ffmpeg", 
        "-ss", str(start_time),  # Set the start time
        "-i", input_file,  # Input file
        "-t", str(duration),  # Duration of the clip
        "-r", str(frame_rate),  # Set the frame rate
        "-c:v", "libx264",  # Video codec
        "-crf", "18",  # Constant rate factor for quality
        "-preset", "fast",  # Encoding preset
        "-an",  # Remove audio
        output_file  # Output file
    ]
    
    # Run the command
    subprocess.run(ffmpeg_command, check=True)
# Run the precise clipping function
precise_clip(input_file, start_time, duration, output_file, frame_rate)
